In [1]:
from pyspark.sql import SparkSession, SQLContext

# initiate the Spark Session
ss = SparkSession.builder.getOrCreate()

In [ ]:
# Read your csv data file whichever path it lies in
mfcc_df = ss.read.csv("s3://pysparkml-for-audio/mfcc_final.csv", header=True, inferSchema=True).cache()
# Remove unnecessary column
mfcc_df = mfcc_df.drop('_co')

# define a udf that extracts the speaker id out of the filename
from pyspark.sql.functions import *
def get_speaker(filename):
    i = filename.index('sp')
    return int(filename[i+2:i+6])
speaker = udf(get_speaker)

mfcc = mfcc_df.withColumn('Speaker', speaker('FileName')).drop('FileName')

speaker_count = mfcc.groupBy("Speaker").count().orderBy("count", ascending=True)
# speaker5 = results.join(speaker_id, "Speaker",'left_semi')
# speaker5_rdd = speaker5.rdd

average_num_wav_files = speaker_count.select(avg("count")).rdd.first()[0]
std_dev_num_wav_files = speaker_count.select(stddev_pop("count")).rdd.first()[0]
min_num_wav_files = int(average_num_wav_files - std_dev_num_wav_files)
max_num_wav_files = int(average_num_wav_files + std_dev_num_wav_files)

speaker_id = speaker_count.filter("count >= {} and count <= {}".format(min_num_wav_files, max_num_wav_files))
# join on your dataset i.e. mfcc, cqt or both concatenated
speaker_114 = mfcc.join(speaker_id, "Speaker",'left_semi')